In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [28]:
df = df.drop(labels=['id'], axis=1)

In [29]:
#Independent and dependent features
X = df.drop(labels=['price'], axis=1)
y = df['price']

In [30]:
#Segregrating numerical and categorical columns
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [31]:
#define the ranking 
cut_categories = ['Fair','Good','Very Good', 'Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [33]:
#numerical pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

#categorical pipeline
cat_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])


In [34]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=30)

In [35]:
X_test

,carat,cut,color,clarity,depth,table,x,y,z
70432,0.53,Premium,E,VS2,60.8,56.0,5.24,5.21,3.19
64839,0.71,Very Good,H,SI1,62.9,57.0,5.67,5.69,3.56
185316,0.30,Ideal,H,IF,62.1,57.0,4.27,4.29,2.66
84658,1.24,Premium,G,VS2,61.6,61.0,6.88,6.82,4.21
31953,0.36,Premium,E,VS1,60.4,58.0,4.60,4.63,2.80
...,...,...,...,...,...,...,...,...,...
192694,1.26,Very Good,H,VS2,63.1,56.0,6.83,6.87,4.33
133935,0.33,Ideal,F,VVS1,62.1,55.0,4.45,4.48,2.77
44763,0.70,Premium,J,VS2,58.2,60.0,5.83,5.80,3.38
29856,0.41,Ideal,G,SI1,61.6,57.0,4.77,4.75,2.93


In [36]:
# col_name = ['num_pipeline_carat', 'num_pipeline_depth', 'num_pipeline_table','num_pipeline_x', 'num_pipeline_y', 'num_pipeline_z', 'cat_pipeline_cut', 'cat_pipeline_color', 'cat_pipeline_clarity']

In [37]:
# col_name1 = ['carat','cut','color','clarity','depth','table','x','y','z']

In [38]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [39]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [40]:
## Model Training

In [41]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [42]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [43]:
regressor.coef_

array([ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
        -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
         650.76431652])

In [44]:
regressor.intercept_

3970.7662895476774

In [45]:
import numpy as np

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    score = r2_score(true, predicted)
    
    return mae, rmse, score

In [47]:
#Train Multiple Models

models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mae, rmse, score = evaluate_model(y_test, y_pred)
    
    print(list(models.values())[i])
    model_list.append(list(models.values())[i])
    r2_list.append(score)
    
    print("Model Taining Performance")
    print(f"RMSE = {rmse}")
    print(f"MAE = {mae}")
    print(f"Score = {score*100}")
    print("\n")
    

LinearRegression()
Model Taining Performance
RMSE = 1013.9047094344002
MAE = 674.0255115796855
Score = 93.68908248567512


Lasso()
Model Taining Performance
RMSE = 1013.8784226767013
MAE = 675.0716923362158
Score = 93.68940971841704


Ridge()
Model Taining Performance
RMSE = 1013.9059272772382
MAE = 674.0555800810192
Score = 93.68906732505846


ElasticNet()
Model Taining Performance
RMSE = 1533.4162456064046
MAE = 1060.7368759154729
Score = 85.56494831165182


